In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from datetime import datetime, timedelta

In [2]:
light_on_Merged_FullDataSet = pd.read_csv('../data/MergedData.csv')
light_on = light_on_Merged_FullDataSet #.head(9000)

/var/folders/nl/hqcw9pqd0ns4q0b61syv0q2m0000gn/T/ipykernel_7238/2186112238.py:1: DtypeWarning: Columns (16,22,23,26,27,29,30,31,32,33,34,35,36,38,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  light_on_Merged_FullDataSet = pd.read_csv('../data/MergedData.csv')


convert columns to appropriate format

In [3]:
cols = ['spn', 'fmi', 'AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 'Speed', 'Throttle', 'TurboBoostPressure']
light_on[cols] = light_on[cols].apply(pd.to_numeric, errors='coerce', axis=1)
light_on

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,NaN,False,78.8,1023.0,True,NaN,0.00000,3276.75,NaN,0.00
1,1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,...,NaN,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,NaN,NaN,NaN,16639.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061319,1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,...,NaN,NaN,NaN,1023.0,NaN,NaN,NaN,NaN,NaN,NaN
1061320,1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,...,32.0,True,98.6,18431.0,False,NaN,65.01096,NaN,73.2,7.83
1061321,1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,...,NaN,True,91.4,17407.0,NaN,NaN,66.57410,NaN,100.0,6.96
1061322,1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,...,NaN,True,100.4,1023.0,False,NaN,11.84489,14.1,100.0,1.74


In [4]:
light_on['spn_fmi'] = light_on['spn'].astype(str) + '_' + light_on['fmi'].astype(str)
light_on

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,False,78.8,1023.0,True,NaN,0.00000,3276.75,NaN,0.00,111.0_17.0
1,1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,...,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,629.0_12.0
2,2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,1807.0_2.0
3,3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,1807.0_2.0
4,4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,NaN,NaN,16639.0,NaN,NaN,NaN,NaN,NaN,NaN,4364.0_17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061319,1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,...,NaN,NaN,1023.0,NaN,NaN,NaN,NaN,NaN,NaN,1761.0_17.0
1061320,1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,...,True,98.6,18431.0,False,NaN,65.01096,NaN,73.2,7.83,1569.0_31.0
1061321,1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,...,True,91.4,17407.0,NaN,NaN,66.57410,NaN,100.0,6.96,3216.0_10.0
1061322,1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,...,True,100.4,1023.0,False,NaN,11.84489,14.1,100.0,1.74,111.0_18.0


In [5]:
# dummy_columns = pd.get_dummies(light_on, columns = ['spn_fmi']) # temporarily commented out in order to reduce file size by removing the dummy columns 
dummy_columns = light_on

In [6]:
dummy_columns['is_fullderate'] = (dummy_columns['spn'] == 5246).astype(int)

In [7]:
dummy_columns = dummy_columns.sort_values(['EquipmentID', 'EventTimeStamp'])
dummy_columns['is_fullderate_group'] = dummy_columns.groupby('EquipmentID')['is_fullderate'].cumsum()

In [8]:
Num_of_FullDerates_inDataSet = dummy_columns['is_fullderate_group'].sum()
Num_of_FullDerates_inDataSet # 1

np.int64(393460)

In [9]:
dummy_columns['EquipID_Index'] = dummy_columns['EquipmentID'].astype(str) + '_' + dummy_columns['is_fullderate_group'].astype(str)

dummy_columns.to_csv('../data/TwentyRows_Preceding_SPN5246_LightOn_Truck_Identifier_SubsequentTrialRun.csv')

In [ ]:
def check_time_intervals(df, EventTimeStamp='EventTimeStamp', condition_column='spn', condition_value=5246, interval_column_name='time_to_next_SPN5246'):
    """
    Calculate the time interval between the current row and the next row(s) that meet a specific condition.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the timestamp and condition columns.
    EventTimeStamp : str
        The name of the column containing datetime values.
    condition_column : str, optional
        The name of the column to check for the condition. If None, all rows are considered.
    condition_value : any, optional
        The value to check for in the condition column. If None, all rows are considered.
    interval_column_name : str, optional
        The name for the new column that will store the time interval information.
        Default is 'time_to_next_condition'.
    
    Returns:
    --------
    pandas.DataFrame
        The original dataframe with an additional column showing the time interval
        to the next row meeting the condition.
    """
    # Ensure the timestamp column is in datetime format
    df[EventTimeStamp] = pd.to_datetime(df[EventTimeStamp])
    
    # Create a copy of the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Add a column to store the time intervals with the specified name
    result_df[interval_column_name] = pd.NaT  # Initialize with NaT (Not a Time) for datetime columns
    #timedelta(days=9999999)  # Initialize with a large timedelta

    
    # Filter rows that meet the condition if a condition is specified
    if condition_column is not None and condition_value is not None:
        condition_rows = df[df[condition_column] == condition_value]
    else:
        condition_rows = df
    
    # For each row in the dataframe
    for i in range(len(df) - 1):
        current_time = df.iloc[i][EventTimeStamp]
        current_EquipmentID = df.iloc[i]['EquipmentID']
        
        # Find the next row that meets the condition and is after the current row
        next_condition_rows = condition_rows[(condition_rows[EventTimeStamp] > current_time) & (condition_rows['EquipmentID'] == current_EquipmentID)]
        
        if not next_condition_rows.empty:
            # Get the earliest next condition time
            next_condition_time = next_condition_rows[EventTimeStamp].min()
            
            # Calculate the time difference
            time_diff = next_condition_time - current_time
            
            # Store the result in the specified column
            result_df.iloc[i, result_df.columns.get_loc(interval_column_name)] = time_diff
    
    return result_df


In [11]:
TimeIntervalColumn_AddedTo_DummyColumns = check_time_intervals(dummy_columns, EventTimeStamp='EventTimeStamp', condition_column='spn', condition_value=5246, interval_column_name='time_to_next_SPN5246') 
TimeIntervalColumn_AddedTo_DummyColumns

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246
43931,48303,49415,2363162,2015-05-11 13:11:20,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,4.475814,3276.75,14.4,0.58,639.0_2.0,0,0,301_0,428 days 05:59:47
45693,50251,51363,2400445,2015-05-13 08:22:32,Condition Exists Cruise Control Enable Switch,NaN,unknown,unknown,unknown,unknown,...,NaN,64.637170,3276.75,89.6,4.64,596.0_31.0,0,0,301_0,426 days 10:48:35
50271,55260,57330,2482983,2015-05-18 09:34:05,Abnormal Rate of Change Aftertreatment 1 Outle...,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,...,NaN,63.525490,3276.75,0.0,20.30,3226.0_10.0,0,0,301_0,421 days 09:37:02
54326,59636,61706,2554027,2015-05-21 13:57:35,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,4.364162,3276.75,12.0,1.45,639.0_2.0,0,0,301_0,418 days 05:13:32
54418,59731,61801,2555325,2015-05-21 14:54:32,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,NaN,639.0_2.0,0,0,301_0,418 days 04:16:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4036,4494,4495,1052146,2015-02-24 16:24:05,Low (Severity Medium) Catalyst Tank Level,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,NaN,1761.0_18.0,0,0,R1762_0,9999999 days 00:00:00
5907,6438,6439,1089561,2015-02-26 13:12:11,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,2.058292,3276.75,0.0,0.29,5848.0_9.0,0,0,R1762_0,9999999 days 00:00:00
5915,6446,6447,1090499,2015-02-26 13:50:59,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,NaN,5848.0_9.0,0,0,R1762_0,9999999 days 00:00:00
4472,4952,4953,1059704,2015-02-25 06:08:43,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,4.378725,3276.75,0.0,1.74,639.0_2.0,0,0,R1764_0,9999999 days 00:00:00


In [12]:
def convert_to_hours(delta):
    total_seconds = delta.dt.total_seconds()
    hours = (total_seconds / 3600).astype(float)
    return hours


In [13]:
TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] = convert_to_hours(TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246']) 
TimeIntervalColumn_AddedTo_DummyColumns

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246
43931,48303,49415,2363162,2015-05-11 13:11:20,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,4.475814,3276.75,14.4,0.58,639.0_2.0,0,0,301_0,1.027800e+04
45693,50251,51363,2400445,2015-05-13 08:22:32,Condition Exists Cruise Control Enable Switch,NaN,unknown,unknown,unknown,unknown,...,NaN,64.637170,3276.75,89.6,4.64,596.0_31.0,0,0,301_0,1.023481e+04
50271,55260,57330,2482983,2015-05-18 09:34:05,Abnormal Rate of Change Aftertreatment 1 Outle...,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,...,NaN,63.525490,3276.75,0.0,20.30,3226.0_10.0,0,0,301_0,1.011362e+04
54326,59636,61706,2554027,2015-05-21 13:57:35,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,4.364162,3276.75,12.0,1.45,639.0_2.0,0,0,301_0,1.003723e+04
54418,59731,61801,2555325,2015-05-21 14:54:32,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,NaN,639.0_2.0,0,0,301_0,1.003628e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4036,4494,4495,1052146,2015-02-24 16:24:05,Low (Severity Medium) Catalyst Tank Level,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,NaN,1761.0_18.0,0,0,R1762_0,2.400000e+08
5907,6438,6439,1089561,2015-02-26 13:12:11,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,2.058292,3276.75,0.0,0.29,5848.0_9.0,0,0,R1762_0,2.400000e+08
5915,6446,6447,1090499,2015-02-26 13:50:59,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,NaN,5848.0_9.0,0,0,R1762_0,2.400000e+08
4472,4952,4953,1059704,2015-02-25 06:08:43,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,4.378725,3276.75,0.0,1.74,639.0_2.0,0,0,R1764_0,2.400000e+08


In [14]:
TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'].min()

np.float64(0.0002777777777777778)

In [ ]:
conditions = [
    TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] <= 0.001,
    (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] <= 2) & (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] > 0.0002777778),
    (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] > 2) & (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] <= 24),
    TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] > 24
]

choices = [3, 2, 1, 0]
TimeIntervalColumn_AddedTo_DummyColumns['time_interval_to_SPN5246_class'] = np.select(conditions, choices)

In [26]:
TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['spn'] == 5246]['spn_fmi'].value_counts()

spn_fmi
5246.0_0.0     622
5246.0_16.0    144
5246.0_15.0    103
5246.0_19.0     61
5246.0_14.0      8
Name: count, dtype: int64

In [25]:
TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['time_interval_to_SPN5246_class'] == 3]['spn_fmi'].value_counts()

spn_fmi
4096.0_31.0    6
4094.0_18.0    4
74.0_14.0      2
3226.0_9.0     2
1761.0_1.0     2
4349.0_7.0     1
1787.0_11.0    1
3698.0_9.0     1
1068.0_2.0     1
3216.0_10.0    1
5246.0_16.0    1
1761.0_9.0     1
Name: count, dtype: int64

In [16]:
Pre2019_df = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] < datetime(2019, 1, 1)]
Pre2019_df

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246,time_interval_to_SPN5246_class
43931,48303,49415,2363162,2015-05-11 13:11:20,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,4.475814,3276.75,14.4,0.58,639.0_2.0,0,0,301_0,1.027800e+04,0
45693,50251,51363,2400445,2015-05-13 08:22:32,Condition Exists Cruise Control Enable Switch,NaN,unknown,unknown,unknown,unknown,...,64.637170,3276.75,89.6,4.64,596.0_31.0,0,0,301_0,1.023481e+04,0
50271,55260,57330,2482983,2015-05-18 09:34:05,Abnormal Rate of Change Aftertreatment 1 Outle...,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,...,63.525490,3276.75,0.0,20.30,3226.0_10.0,0,0,301_0,1.011362e+04,0
54326,59636,61706,2554027,2015-05-21 13:57:35,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,4.364162,3276.75,12.0,1.45,639.0_2.0,0,0,301_0,1.003723e+04,0
54418,59731,61801,2555325,2015-05-21 14:54:32,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,639.0_2.0,0,0,301_0,1.003628e+04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4036,4494,4495,1052146,2015-02-24 16:24:05,Low (Severity Medium) Catalyst Tank Level,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,1761.0_18.0,0,0,R1762_0,2.400000e+08,0
5907,6438,6439,1089561,2015-02-26 13:12:11,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,2.058292,3276.75,0.0,0.29,5848.0_9.0,0,0,R1762_0,2.400000e+08,0
5915,6446,6447,1090499,2015-02-26 13:50:59,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,5848.0_9.0,0,0,R1762_0,2.400000e+08,0
4472,4952,4953,1059704,2015-02-25 06:08:43,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,4.378725,3276.75,0.0,1.74,639.0_2.0,0,0,R1764_0,2.400000e+08,0


In [17]:
Pre2019_df_AltApproach = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] < pd.Timestamp('2019-01-01')]
Pre2019_df_AltApproach

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246,time_interval_to_SPN5246_class
43931,48303,49415,2363162,2015-05-11 13:11:20,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,4.475814,3276.75,14.4,0.58,639.0_2.0,0,0,301_0,1.027800e+04,0
45693,50251,51363,2400445,2015-05-13 08:22:32,Condition Exists Cruise Control Enable Switch,NaN,unknown,unknown,unknown,unknown,...,64.637170,3276.75,89.6,4.64,596.0_31.0,0,0,301_0,1.023481e+04,0
50271,55260,57330,2482983,2015-05-18 09:34:05,Abnormal Rate of Change Aftertreatment 1 Outle...,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,...,63.525490,3276.75,0.0,20.30,3226.0_10.0,0,0,301_0,1.011362e+04,0
54326,59636,61706,2554027,2015-05-21 13:57:35,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,4.364162,3276.75,12.0,1.45,639.0_2.0,0,0,301_0,1.003723e+04,0
54418,59731,61801,2555325,2015-05-21 14:54:32,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,639.0_2.0,0,0,301_0,1.003628e+04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4036,4494,4495,1052146,2015-02-24 16:24:05,Low (Severity Medium) Catalyst Tank Level,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,1761.0_18.0,0,0,R1762_0,2.400000e+08,0
5907,6438,6439,1089561,2015-02-26 13:12:11,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,2.058292,3276.75,0.0,0.29,5848.0_9.0,0,0,R1762_0,2.400000e+08,0
5915,6446,6447,1090499,2015-02-26 13:50:59,NaN,NaN,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,...,NaN,NaN,NaN,NaN,5848.0_9.0,0,0,R1762_0,2.400000e+08,0
4472,4952,4953,1059704,2015-02-25 06:08:43,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,...,4.378725,3276.75,0.0,1.74,639.0_2.0,0,0,R1764_0,2.400000e+08,0


In [18]:
Pre2019_df.to_csv('../data/Pre2019_df.csv')

In [19]:
In2019_df = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] > datetime(2018, 12, 31)]
In2019_df

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246,time_interval_to_SPN5246_class
952775,1062090,1104925,73930906,2019-01-11 12:51:18,High Voltage (Fuel Level),NaN,unknown,unknown,unknown,unknown,...,46.52031,NaN,34.4,13.05,96.0_3.0,0,1,301_1,239999976.0,0
952776,1062091,1104926,73930907,2019-01-11 12:51:18,High Voltage (Left Fuel Level Sensor),NaN,unknown,unknown,unknown,unknown,...,46.52031,NaN,34.4,13.05,829.0_3.0,0,1,301_1,239999976.0,0
952781,1062096,1104931,73934247,2019-01-11 13:11:40,High Voltage (Fuel Level),NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,96.0_3.0,0,1,301_1,239999976.0,0
952782,1062097,1104932,73934248,2019-01-11 13:11:40,High Voltage (Left Fuel Level Sensor),NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,829.0_3.0,0,1,301_1,239999976.0,0
952809,1062135,1104970,73942005,2019-01-11 13:13:38,High Voltage (Fuel Level),NaN,unknown,unknown,unknown,unknown,...,0.00000,NaN,0.0,1.16,96.0_3.0,0,1,301_1,239999976.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977126,1089938,1137548,80503703,2019-03-16 11:22:32,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,AAAI000051*AAAM010056*Z091290 *A82J170202A_b...,6232171647,EC80ESP,BNDWS,...,NaN,NaN,NaN,NaN,639.0_2.0,0,0,105437340_0,239999976.0,0
977150,1089962,1137572,80509396,2019-03-16 11:24:04,Special Instructions Engine Starter Solenoid L...,NaN,5569997 *E250.e003*,K1243673,FAOM-xx810S-EC3,EATON,...,0.00000,NaN,100.0,0.29,1321.0_14.0,0,0,105437340_0,239999976.0,0
977151,1089963,1137573,80509397,2019-03-16 11:27:41,NaN,NaN,5569997 *E250.e003*,K1243673,FAOM-xx810S-EC3,EATON,...,0.00000,NaN,100.0,0.29,520321.0_13.0,0,0,105437340_0,239999976.0,0
978491,1091502,1139112,80907838,2019-03-20 10:57:34,Special Instructions Engine Starter Solenoid L...,NaN,NaN,K1243673,FAOM-xx810S-EC3,EATON,...,NaN,NaN,NaN,NaN,1321.0_14.0,0,0,105437340_0,239999976.0,0


In [20]:
In2019_df_AltApproach = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] > pd.Timestamp('2018-12-31')]
In2019_df_AltApproach

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,time_to_next_SPN5246,time_interval_to_SPN5246_class
952775,1062090,1104925,73930906,2019-01-11 12:51:18,High Voltage (Fuel Level),NaN,unknown,unknown,unknown,unknown,...,46.52031,NaN,34.4,13.05,96.0_3.0,0,1,301_1,239999976.0,0
952776,1062091,1104926,73930907,2019-01-11 12:51:18,High Voltage (Left Fuel Level Sensor),NaN,unknown,unknown,unknown,unknown,...,46.52031,NaN,34.4,13.05,829.0_3.0,0,1,301_1,239999976.0,0
952781,1062096,1104931,73934247,2019-01-11 13:11:40,High Voltage (Fuel Level),NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,96.0_3.0,0,1,301_1,239999976.0,0
952782,1062097,1104932,73934248,2019-01-11 13:11:40,High Voltage (Left Fuel Level Sensor),NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,829.0_3.0,0,1,301_1,239999976.0,0
952809,1062135,1104970,73942005,2019-01-11 13:13:38,High Voltage (Fuel Level),NaN,unknown,unknown,unknown,unknown,...,0.00000,NaN,0.0,1.16,96.0_3.0,0,1,301_1,239999976.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977126,1089938,1137548,80503703,2019-03-16 11:22:32,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,AAAI000051*AAAM010056*Z091290 *A82J170202A_b...,6232171647,EC80ESP,BNDWS,...,NaN,NaN,NaN,NaN,639.0_2.0,0,0,105437340_0,239999976.0,0
977150,1089962,1137572,80509396,2019-03-16 11:24:04,Special Instructions Engine Starter Solenoid L...,NaN,5569997 *E250.e003*,K1243673,FAOM-xx810S-EC3,EATON,...,0.00000,NaN,100.0,0.29,1321.0_14.0,0,0,105437340_0,239999976.0,0
977151,1089963,1137573,80509397,2019-03-16 11:27:41,NaN,NaN,5569997 *E250.e003*,K1243673,FAOM-xx810S-EC3,EATON,...,0.00000,NaN,100.0,0.29,520321.0_13.0,0,0,105437340_0,239999976.0,0
978491,1091502,1139112,80907838,2019-03-20 10:57:34,Special Instructions Engine Starter Solenoid L...,NaN,NaN,K1243673,FAOM-xx810S-EC3,EATON,...,NaN,NaN,NaN,NaN,1321.0_14.0,0,0,105437340_0,239999976.0,0


In [21]:
In2019_df.to_csv('../data/In2019_df.csv')

In [22]:
TimeIntervalColumn_AddedTo_DummyColumns.to_csv('../data/MergedFullDataSet_TimeIntervalLabeled_PerTruck.csv')